# Get the data for single who wants to move to Austin

#### This is on assumption that he/she is looking for shopping nightlide and apartments

In [6]:
#define the Headers 
import folium
import pandas as pd
import numpy as np
import vincent

In [35]:
# Create the Data frame from night life 
nightlife_df = pd.read_csv('data_sources/nightlife_data_combined_with_cords.csv')

nightlife_df = nightlife_df.drop(['Unnamed: 0'], axis=1)

nightlife_df.head()

,ZipCode,Restaurant Count,Restaurant Rating,Bar Count,Bar Rating,Club Count,Club Rating,Lat,Lon,Bar/Restaurant Count,Bar/Restaurant Rating,Total Nightlife Count,Combined Rating
0,73301,3.0,4.500000,0.0,0.000000,0.0,0.000000,30.220098,-97.749046,3.0,9.000000,3.0,9.000000
1,78617,1.0,4.000000,2.0,3.000000,0.0,0.000000,30.201016,-97.665245,3.0,6.666667,3.0,6.666667
2,78646,2.0,4.500000,0.0,0.000000,0.0,0.000000,30.273205,-97.800709,2.0,9.000000,2.0,9.000000
3,78660,4.0,3.125000,2.0,3.000000,0.0,0.000000,30.453564,-97.666576,6.0,6.166667,6.0,6.166667
4,78701,201.0,4.186567,194.0,4.265464,223.0,3.376682,30.267238,-97.741325,618.0,5.401294,618.0,7.838188


In [36]:
nightlife_df = nightlife_df.drop(['Bar/Restaurant Count', 'Bar/Restaurant Rating'], axis=1)
nightlife_df.head()

,ZipCode,Restaurant Count,Restaurant Rating,Bar Count,Bar Rating,Club Count,Club Rating,Lat,Lon,Total Nightlife Count,Combined Rating
0,73301,3.0,4.500000,0.0,0.000000,0.0,0.000000,30.220098,-97.749046,3.0,9.000000
1,78617,1.0,4.000000,2.0,3.000000,0.0,0.000000,30.201016,-97.665245,3.0,6.666667
2,78646,2.0,4.500000,0.0,0.000000,0.0,0.000000,30.273205,-97.800709,2.0,9.000000
3,78660,4.0,3.125000,2.0,3.000000,0.0,0.000000,30.453564,-97.666576,6.0,6.166667
4,78701,201.0,4.186567,194.0,4.265464,223.0,3.376682,30.267238,-97.741325,618.0,7.838188


## Plot the map

In [90]:
def Plotmarkers(mp, df):
    index = ['Restaurants', 'Bars', 'Clubs']
    
    for i, row in df.iterrows():
        #get the bars that needs to be plotted 
        data = {}
        #Generalize the rating to base 10
        data['Rating'] = [nightlife_df.loc[i]['Restaurant Rating']*2, nightlife_df.loc[i]['Bar Rating']* 2, nightlife_df.loc[i]['Club Rating']* 2] 
        
        #Convert the dict into DataFrame
        df_bar = pd.DataFrame(data, index=index)
        
        #Plot a Barchart
        bar_chart = vincent.GroupedBar(df_bar, width=100, height=100 )
        
        #define color x and y axis 
        bar_chart.colors(range_= [ '#8856a7'])        
        x = df.loc[i]["ZipCode"]
        bar_chart.axis_titles(x="Night Life", y='Ratings')
        
        #tilt the y axis
        ax = vincent.AxisProperties(labels = vincent.PropertySet(angle=vincent.ValueRef(value=-45)))
        bar_chart.axes[0].properties = ax
        
        #create a pop up
        popup = folium.Popup()
        folium.Vega(bar_chart, height=150, width=200).add_to(popup)
        #add to the marker
        folium.Marker([df.loc[i]['Lat'], df.loc[i]['Lon']], popup=popup,
                 tooltip=nightlife_df.loc[i]["ZipCode"].astype(int)).add_to(mp) 

    return mp

In [92]:
def Plotmap(mp, df):
    
    # Add marker one by one on the map
    for i, row in df.iterrows():
        #print(f"index {i}")
        folium.Circle(
            location=[df.loc[i]['Lat'], df.loc[i]['Lon']],
            popup=str(df.loc[i]['ZipCode']),
            radius=df.loc[i]['Total Nightlife Count']*5,
            color='crimson',
            fill=True,
            fill_color='crimson'
        ).add_to(mp)  
        
    Plotmarkers(mp, df)
    return mp 
#Plotmap(nightlife_df)


## Call the functions to plot the graph

In [95]:
#define Austin map Lat Lon[30.35, -97.75]
mp = folium.Map(location=[30.35, -97.75], zoom_start=11)

#choice could be anything for this notebook it is only Nightlife
choice = 'Nightlife'
if choice == 'Nightlife':
    Plotmap(mp, nightlife_df)
else:
    print("Nightlife")
    
mp.save('SingleChart.html')
mp
    